In [66]:
import pandas as pd
import numpy as np
import datetime
from openpyxl import load_workbook

In [91]:
SEASON = 12
MATCHDAY = 20

In [92]:
mc = pd.read_excel('data/match_'+str(MATCHDAY)+'.xlsx')
mc.drop('№', axis=1, inplace=True)
mc.rename(columns={'index': 'Name'}, inplace=True)

In [93]:
mc.columns

Index(['Name', 'Итог', 'манчестер юнайтед – ливерпуль', '2\n4',
       'ювентус – интер', '3\n2', 'бенфика – спортинг', '4\n3', 'рома – лацио',
       '2\n0', 'брайтон – вест хэм', '1\n1', 'динамо – цска', '3\n2.1',
       'ростов – краснодар', '1\n3', 'вильярреал – севилья', '4\n0',
       'рб лейпциг – вольфсбург', '2\n2', 'челси – лестер', '2\n1'],
      dtype='object')

In [94]:
#Заголовки нулевой таблицы для создания ДБ ставок
zeros_col = ['Total Points', 'Game Points', 'Bomb Points', 'Exact Score', 'Right Margin', 'Right Result', 'Wrong Result', 'Uniques']

matches = pd.read_excel('output/season_'+str(SEASON)+'.xlsx', sheet_name='matches', index_col=0)
players = pd.read_excel('output/season_'+str(SEASON)+'.xlsx', sheet_name='players', index_col=0)
season = pd.read_excel('output/season_'+str(SEASON)+'.xlsx', sheet_name='season', index_col=0)
teams = pd.read_excel('output/season_'+str(SEASON)+'.xlsx', sheet_name='teams', index_col=0)

In [95]:
def proceed_bet(player_id, match_id):
    """
    proceed_bet(player_id, match_id)
    Сверяет ставки с результатом матча.
    player_id = Id участника
    match_id = Id матча
    """
    player_name = players['Name'].iloc[player_id]
    
    h_team = matches['Hometeam'].loc[match_id]
    a_team = matches['Awayteam'].loc[match_id]
    match = str(h_team)+' - '+str(a_team)
    
    try:
        hscore_true = matches['H_score'].loc[match_id]
        ascore_true = matches['A_score'].loc[match_id]

        hscore_bet = int(bets['bet_hscore_'+str(match_id)].loc[player_id])
        ascore_bet = int(bets['bet_ascore_'+str(match_id)].loc[player_id])
        
    except ValueError:
        print(player_name, 'made no bet on', match)
        bets['points_'+str(match_id)].loc[player_id] = 0
        return
    
    true_score= str(hscore_true)+':'+str(ascore_true)
    bet_score= str(hscore_bet)+':'+str(ascore_bet)
    
    points_granted = 0
    
    if (hscore_true == hscore_bet and ascore_true == ascore_bet):
        points_granted = 5
        matches['Exact Score'].loc[match_id] += 1
        season['Exact Score'].loc[player_id] += 1
        bets['Exact Score'].loc[player_id] += 1
    elif (hscore_true - ascore_true) == (hscore_bet - ascore_bet):
        points_granted = 3
        matches['Right Margin'].loc[match_id] += 1
        season['Right Margin'].loc[player_id] += 1
        bets['Right Margin'].loc[player_id] += 1
    elif ((hscore_true > ascore_true) and (hscore_bet > ascore_bet)) or ((hscore_true < ascore_true) and (hscore_bet < ascore_bet)):
        points_granted = 2
        matches['Right Result'].loc[match_id] += 1
        season['Right Result'].loc[player_id] += 1
        bets['Right Result'].loc[player_id] += 1
    else:
        matches['Wrong Result'].loc[match_id] += 1
        season['Wrong Result'].loc[player_id] += 1
        bets['Wrong Result'].loc[player_id] += 1
    matches['Bets'].loc[match_id] += 1
    season['Bets'].loc[player_id] += 1
    bets['points_'+str(match_id)].loc[player_id] = points_granted
    return

def count_total(player_id):
    """
    Считает суммарное число баллов, суммируя значения с bets 
    """
    total = 0
    for i in range(11, bets.shape[1], 3):
        total += bets.loc[player_id].iloc[i]
    bets['Game Points'].loc[player_id] = total
    season['Game Points'].loc[player_id] += total
    print(bets.iloc[player_id, 0], total)
    
def find_uniques(match_ids):
    h_score = str(matches['H_score'].loc[match_ids])
    a_score = str(matches['A_score'].loc[match_ids])
    player_id = bets[bets['bet_hscore_'+str(match_ids)] == h_score][bets['bet_ascore_'+str(match_ids)] == a_score].index[0]
    bets['Uniques'].loc[player_id] += 1
    season['Uniques'].loc[player_id] += 1
    matches['Uniques'].iloc[match_ids] += 1
    return player_id

def get_match_ids(season=SEASON, matchday=MATCHDAY):
    ids = matches[matches['Season'] == season][matches['Matchday'] == matchday].index
    return_list = ['Name', 'Total Points', 'Game Points', 'Bomb Points', 'Exact Score', 'Right Margin', 'Right Result', 'Wrong Result', 'Uniques']
    for i in ids:
        return_list.append('bet_hscore_'+str(i))
        return_list.append('bet_ascore_'+str(i))
        return_list.append('points_'+str(i))
    return return_list

In [96]:
for match, score in zip(mc.columns[2::2], mc.columns[3::2]):
    hometeam, awayteam = match.split(' – ')[0], match.split(' – ')[1]
    h_score, a_score = score.split('\n')[0], score.split('\n')[1][0]
      
    if matches[matches['Matchday'] == MATCHDAY][matches['Hometeam'] == hometeam][matches['Awayteam'] == awayteam].shape[0] > 0:
        print(f'{hometeam} - {awayteam} уже есть')
    else:
        matches = matches.append({'Hometeam': hometeam, 'Awayteam': awayteam,
                                  'H_score': int(h_score), 'A_score': int(a_score),
                                  'Season': SEASON, 'Matchday': MATCHDAY, 'Date': datetime.datetime.now(), 'Bets': 0,
                                  'Exact Score': 0, 'Right Margin': 0, 'Right Result': 0, 
                                  'Wrong Result': 0, 'Uniques': 0}, ignore_index=True)
    
bets_columns = get_match_ids(SEASON,MATCHDAY)
bets = pd.DataFrame(pd.concat([mc['Name'], pd.DataFrame(np.zeros((mc['Name'].shape[0],8)), columns=zeros_col)], axis=1), columns=bets_columns)  

#Очень тупо, я повторяю цикл, потому что между этими операциями ме надо добавить БД для ставок
for match, score in zip(mc.columns[2::2], mc.columns[3::2]):
    hometeam, awayteam = match.split(' – ')[0], match.split(' – ')[1]
    h_score, a_score = score.split('\n')[0], score.split('\n')[1][0]    
    match_id = matches[matches['Hometeam'] == hometeam][matches['Awayteam'] == awayteam].index[0]   
    
    bets['bet_hscore_'+str(match_id)] = mc[match].str.split(':').str[0]
    bets['bet_ascore_'+str(match_id)] = mc[match].str.split(':').str[1]

<ipython-input-96-7dd6f4ee2aab>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if matches[matches['Matchday'] == MATCHDAY][matches['Hometeam'] == hometeam][matches['Awayteam'] == awayteam].shape[0] > 0:
<ipython-input-96-7dd6f4ee2aab>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if matches[matches['Matchday'] == MATCHDAY][matches['Hometeam'] == hometeam][matches['Awayteam'] == awayteam].shape[0] > 0:
<ipython-input-96-7dd6f4ee2aab>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if matches[matches['Matchday'] == MATCHDAY][matches['Hometeam'] == hometeam][matches['Awayteam'] == awayteam].shape[0] > 0:
<ipython-input-96-7dd6f4ee2aab>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if matches[matches['Matchday'] == MATCHDAY][matches['Hometeam'] == hometeam][matches['Awayteam'] == awayteam].shape[0] > 0:
<ipython-input-96-7dd6f4ee2aab>:5: UserWarning: Bool

In [97]:
for player in range(0,players['Name'].shape[0]):
    for match in matches[matches['Season']==SEASON][matches['Matchday']==MATCHDAY].index:
        proceed_bet(player,match)

H:\dev\py\python-3.8.5.amd64\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
H:\dev\py\python-3.8.5.amd64\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
H:\dev\py\python-3.8.5.amd64\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ver

Белькин Сергей made no bet on манчестер юнайтед - ливерпуль
Белькин Сергей made no bet on ювентус - интер
Белькин Сергей made no bet on бенфика - спортинг
Белькин Сергей made no bet on рома - лацио
Белькин Сергей made no bet on брайтон - вест хэм
Белькин Сергей made no bet on динамо - цска
Белькин Сергей made no bet on ростов - краснодар
Белькин Сергей made no bet on вильярреал - севилья
Белькин Сергей made no bet on рб лейпциг - вольфсбург
Белькин Сергей made no bet on челси - лестер
Гришин Антон made no bet on манчестер юнайтед - ливерпуль
Зубатов Михаил made no bet on манчестер юнайтед - ливерпуль
Колодин Дмитрий made no bet on манчестер юнайтед - ливерпуль
Колодин Дмитрий made no bet on ювентус - интер
Колодин Дмитрий made no bet on бенфика - спортинг
Колодин Дмитрий made no bet on рома - лацио
Колодин Дмитрий made no bet on брайтон - вест хэм
Колодин Дмитрий made no bet on динамо - цска
Колодин Дмитрий made no bet on ростов - краснодар
Колодин Дмитрий made no bet on вильярреал - с

H:\dev\py\python-3.8.5.amd64\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
H:\dev\py\python-3.8.5.amd64\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
H:\dev\py\python-3.8.5.amd64\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ver

In [74]:
mc

,Name,Итог,манчестер сити – псж,2\n0,челси – реал мадрид,2\n0.1,арсенал – вильярреал,0\n0,рома – манчестер юнайтед,3\n2,...,галатасарай – бешикташ,3\n1.1,сент\nэтьен – марсель,1\n0,вест хэм – эвертон,0\n1,ювентус – милан,0\n3,реал мадрид – севилья,2\n2
0,Алферов Ян,4.0,2:1,2.0,2:1,2.0,2:1,0.0,1:2,0.0,...,1:2,0.0,0:1,0.0,1:0,0.0,2:1,0.0,2:1,0.0
1,Андреев Иван,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Афанасьев Владимир,2.0,1:2,0.0,2:1,2.0,2:1,0.0,1:1,0.0,...,1:1,0.0,1:1,0.0,1:1,0.0,2:1,0.0,2:1,0.0
3,Белькин Сергей,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Гришин Антон,7.0,2:1,2.0,2:1,2.0,1:1,3.0,1:2,0.0,...,1:1,0.0,1:2,0.0,2:1,0.0,2:1,0.0,1:1,5.0
5,Данилов Евгений,0.0,2:2,0.0,1:2,0.0,1:2,0.0,0:2,0.0,...,1:2,0.0,1:1,0.0,2:0,0.0,2:1,0.0,2:0,0.0
6,Еременко Владислав,2.0,1:3,0.0,1:0,2.0,1:2,0.0,2:4,0.0,...,1:2,0.0,1:2,0.0,2:1,0.0,1:1,0.0,2:1,0.0
7,Жигалов Александр,4.0,1:0,2.0,1:0,2.0,2:1,0.0,0:3,0.0,...,0:1,0.0,0:2,0.0,2:1,0.0,1:0,0.0,2:1,0.0
8,Зубатов Михаил,0.0,0:1,0.0,1:2,0.0,1:2,0.0,1:3,0.0,...,2:2,0.0,1:2,0.0,2:1,0.0,2:0,0.0,1:2,0.0
9,Колодин Дмитрий,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [98]:
bets['Name']

0             Алферов Ян
1           Андреев Иван
2     Афанасьев Владимир
3         Белькин Сергей
4           Гришин Антон
5        Данилов Евгений
6     Еременко Владислав
7      Жигалов Александр
8         Зубатов Михаил
9        Колодин Дмитрий
10        Конаков Никита
11       Котов Александр
12    Криеванс Владислав
13      Кучеренко Никита
14      Лесик Константин
15          Малевич Егор
16       Махмудов Руслан
17           Минеев Иван
18         Муратов Игорь
19        Невский Леонид
20     Новиков Александр
21        Оксанич Кирилл
22     Сибиряков Георгий
23       Титаренко Антон
24      Фёдоров Владимир
25     Чапаров Александр
26          Шевчук Антон
Name: Name, dtype: object

In [76]:
def find_match_id(tour):
    return 11+(tour-1)*3

find_match_id

In [79]:
bets.iloc[4,find_match_id(14)] = 5


In [99]:
for i in matches[matches['Season']==SEASON][matches['Matchday']==MATCHDAY][matches['Exact Score']==1].index:
    player_id = find_uniques(i)
    bets['points_'+str(i)].loc[player_id] += 1
    
for player in range(0,players['Name'].shape[0]):
    count_total(player)

Алферов Ян 7.0
Андреев Иван 8.0
Афанасьев Владимир 15.0
Белькин Сергей 0.0
Гришин Антон 16.0
Данилов Евгений 20.0
Еременко Владислав 5.0
Жигалов Александр 10.0
Зубатов Михаил 13.0
Колодин Дмитрий 0.0
Конаков Никита 9.0
Котов Александр 2.0
Криеванс Владислав 8.0
Кучеренко Никита 6.0
Лесик Константин 16.0
Малевич Егор 12.0
Махмудов Руслан 5.0
Минеев Иван 0.0
Муратов Игорь 5.0
Невский Леонид 13.0
Новиков Александр 14.0
Оксанич Кирилл 12.0
Сибиряков Георгий 13.0
Титаренко Антон 13.0
Фёдоров Владимир 10.0
Чапаров Александр 6.0
Шевчук Антон 8.0


<ipython-input-99-c524e9ee286a>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  for i in matches[matches['Season']==SEASON][matches['Matchday']==MATCHDAY][matches['Exact Score']==1].index:
H:\dev\py\python-3.8.5.amd64\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


### Bombers

In [100]:
bombs = pd.read_excel('output/bombs_bets.xlsx', index_col=0, sheet_name=str(MATCHDAY))


In [101]:
# Сгрупировал таблицу в формате имя - очки и присвоил в новую переменную
bombs_points = bombs[bombs['Points']>0].groupby(['Name']).sum().sort_values(by=['Points'], ascending=False)

# Теперь итерируем по всей таблице bets. Если имя есть в bomb_points обновляем значение таблицы Bomb Points. 
# Важно вызывать через loc чтобы не вызывать копию, а работать напрямую с табилцей
print('Очки полученные за бомбардиров')
for row in np.arange(bets.shape[0]):
    try:
        bets.loc[row,'Bomb Points'] = bombs_points['Points'][bets.loc[row,'Name']]
        print(bets.loc[row,'Bomb Points'], bets.loc[row,'Name'])
    except KeyError:
        pass

Очки полученные за бомбардиров
4.0 Алферов Ян
4.0 Андреев Иван
2.0 Данилов Евгений
2.0 Еременко Владислав
6.0 Жигалов Александр
4.0 Зубатов Михаил
6.0 Котов Александр
8.0 Криеванс Владислав
2.0 Лесик Константин
4.0 Сибиряков Георгий
6.0 Титаренко Антон
4.0 Фёдоров Владимир
4.0 Чапаров Александр


In [102]:
bets['Total Points'] = bets['Game Points'] + bets['Bomb Points']

season['Bomb Points'] += bets['Bomb Points']
season['Total Points'] = season['Game Points'] + season['Bomb Points']

In [103]:
bets.sort_values(by=['Total Points'], ascending=False)[['Name','Total Points']]

,Name,Total Points
5,Данилов Евгений,22.0
23,Титаренко Антон,19.0
14,Лесик Константин,18.0
22,Сибиряков Георгий,17.0
8,Зубатов Михаил,17.0
4,Гришин Антон,16.0
12,Криеванс Владислав,16.0
7,Жигалов Александр,16.0
2,Афанасьев Владимир,15.0
20,Новиков Александр,14.0


In [105]:
season.sort_values(by=['Total Points'], ascending=False)[['Name','Total Points']]

,Name,Total Points
21,Оксанич Кирилл,373.0
14,Лесик Константин,353.0
22,Сибиряков Георгий,351.0
19,Невский Леонид,343.0
15,Малевич Егор,342.0
4,Гришин Антон,337.0
13,Кучеренко Никита,334.0
12,Криеванс Владислав,333.0
18,Муратов Игорь,320.0
11,Котов Александр,316.0


In [106]:
for match_id in matches[matches['Season']==SEASON][matches['Matchday']==MATCHDAY].index:
    match = matches.iloc[match_id,:]
    hometeam, awayteam = match[0], match[1]
    m_s = matches.iloc[match_id,:]    #Match_series
    for team in [hometeam, awayteam]:
        # Какое говно)))))
        if team == m_s['Hometeam']:
            if m_s['H_score'] > m_s['A_score']:
                win, draw, loss = 1,0,0
            elif m_s['H_score'] < m_s['A_score']:
                win, draw, loss = 0,0,1
            else:
                win, draw, loss = 0,1,0
        elif team == m_s['Awayteam']:
            if m_s['H_score'] > m_s['A_score']:
                win, draw, loss = 0,0,1
            elif m_s['H_score'] < m_s['A_score']:
                win, draw, loss = 1,0,0
            else:
                win, draw, loss = 0,1,0
                      
        if team in teams['Team'].values:
            team_index = teams[teams['Team'] == team].index[0]
            teams.iloc[team_index,2] += 1
            teams.iloc[team_index,3] += win
            teams.iloc[team_index,4] += draw
            teams.iloc[team_index,5] += loss
            teams.iloc[team_index,6] += m_s['Bets']
            teams.iloc[team_index,7] += m_s['Exact Score']
            teams.iloc[team_index,8] += m_s['Right Margin']
            teams.iloc[team_index,9] += m_s['Right Result']
            teams.iloc[team_index,10] += m_s['Wrong Result']
            teams.iloc[team_index,11] += m_s['Uniques']
        else:
            teams = teams.append({'Team': team, 'Matches': 1, 'Wins': win, 'Draws': draw, 'Losses': loss,
                                 'Bets': m_s['Bets'], 'Exact Score': m_s['Exact Score'], 'Right Margin': m_s['Right Margin'], 
                                 'Right Result': m_s['Right Result'], 'Wrong Result': m_s['Wrong Result'], 
                                 'Uniques': m_s['Uniques']}, ignore_index=True)

In [107]:
teams

,Team,Country,Matches,Wins,Draws,Losses,Bets,Exact Score,Right Margin,Right Result,Wrong Result,Uniques
0,леванте,NaN,1,0,1,0,25,1,5,0,19,1
1,гранада,NaN,6,1,1,4,154,30,7,51,66,2
2,майнц,NaN,2,1,1,0,50,5,2,1,42,1
3,унион берлин,NaN,5,2,2,1,125,20,8,3,94,1
4,ньюкасл юнайтед,NaN,1,1,0,0,27,0,4,3,20,0
...,...,...,...,...,...,...,...,...,...,...,...,...
108,башакшехир,NaN,1,0,1,0,24,0,7,0,17,0
109,спортинг,NaN,2,1,0,1,48,1,15,8,24,1
110,динамо,NaN,2,1,0,1,45,0,5,9,31,0
111,сент\nэтьен,NaN,2,2,0,0,46,1,3,1,41,1


In [108]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
book = load_workbook('output/bets.xlsx')
writer = pd.ExcelWriter('output/bets.xlsx', engine='openpyxl')
writer.book = book

# Write each dataframe to a different worksheet.
bets.to_excel(writer, sheet_name=str(MATCHDAY))

writer.save()
writer.close()

In [109]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(f'output/season_{str(SEASON)}.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
season.to_excel(writer, sheet_name='season')
matches.to_excel(writer, sheet_name='matches')
players.to_excel(writer, sheet_name='players')
teams.to_excel(writer, sheet_name='teams')
#scorers.to_excel(writer, sheet_name='scorers_18')

writer.save()